# Creating surface forcing

In [1]:
import os
from roms_tools import Grid, SurfaceForcing     
from datetime import datetime
outputdir = "/home/ctroupin/data/CROCO/"

As always, the first step is to create our grid. Let's start with a coarse-resolution grid with a horizontal resolution of 2000km/100 = 20km.

In [2]:
largegrid = Grid(
    nx=442,  # number of grid points in x-direction
    ny=400,  # number of grid points in y-direction
    size_x=3300,  # domain size in x-direction (in km)
    size_y=3000,  # domain size in y-direction (in km)
    center_lon=-23.,  # longitude of the center of the domain
    center_lat=37.2,  # latitude of the center of the domain
    rot=-5.5,  # rotation of the grid (in degrees)
    N=32,  # number of vertical layers
    verbose=True,
)

2026-01-08 14:13:04 - INFO - === Creating the horizontal grid ===
2026-01-08 14:13:04 - INFO - Total time: 0.136 seconds
2026-01-08 14:13:04 - INFO - ================================================================================================
2026-01-08 14:13:04 - INFO - === Deriving the mask from coastlines ===
2026-01-08 14:13:05 - INFO - Total time: 0.381 seconds
2026-01-08 14:13:05 - INFO - ================================================================================================
2026-01-08 14:13:05 - INFO - === Generating the topography using ETOPO5 data and hmin = 5.0 meters ===
2026-01-08 14:13:05 - INFO - Reading the topography data: 0.033 seconds
2026-01-08 14:13:05 - INFO - Regridding the topography: 0.014 seconds
2026-01-08 14:13:05 - INFO - Domain-wide topography smoothing: 0.035 seconds
2026-01-08 14:13:09 - INFO - Local topography smoothing: 4.758 seconds
2026-01-08 14:13:09 - INFO - Total time: 4.854 seconds
2026-01-08 14:13:09 - INFO - ========================

In [3]:
grid = Grid(
    nx=520,  # number of grid points in x-direction
    ny=450,  # number of grid points in y-direction
    size_x=1560,  # domain size in x-direction (in km)
    size_y=1350,  # domain size in y-direction (in km)
    center_lon=-17.75,  # longitude of the center of the domain
    center_lat=30,  # latitude of the center of the domain
    rot=-15,  # rotation of the grid (in degrees)
    N=40,  # number of vertical layers
    verbose=True,
)

2026-01-08 14:13:09 - INFO - === Creating the horizontal grid ===
2026-01-08 14:13:10 - INFO - Total time: 0.171 seconds
2026-01-08 14:13:10 - INFO - ================================================================================================
2026-01-08 14:13:10 - INFO - === Deriving the mask from coastlines ===
2026-01-08 14:13:10 - INFO - Total time: 0.330 seconds
2026-01-08 14:13:10 - INFO - ================================================================================================
2026-01-08 14:13:10 - INFO - === Generating the topography using ETOPO5 data and hmin = 5.0 meters ===
2026-01-08 14:13:10 - INFO - Reading the topography data: 0.020 seconds
2026-01-08 14:13:10 - INFO - Regridding the topography: 0.015 seconds
2026-01-08 14:13:10 - INFO - Domain-wide topography smoothing: 0.048 seconds
2026-01-08 14:13:14 - INFO - Local topography smoothing: 3.749 seconds
2026-01-08 14:13:14 - INFO - Total time: 3.839 seconds
2026-01-08 14:13:14 - INFO - ========================

Next, we specify the temporal range that we want to make the surface forcing for.

In [4]:
start_time = datetime(2013, 1, 1)
end_time = datetime(2013, 3, 31)

`ROMS-Tools` can create two types of surface forcing:

* physical surface forcing like 10m wind, shortwave radiation, and air temperature at 2m
* biogeochemical (BGC) surface forcing like atmospheric pCO2

Unlike initial conditions data, ROMS can read multiple surface forcing files, so we create these two types separately in the following sections.

## Physical surface forcing

In this section, we use ERA5 data to construct the physical surface forcing. There are two ways to access the data:

- **Stream directly from the cloud** (no download required)
- **Use locally pre-downloaded files** by specifying the path

**Streaming** is convenient: there's no need to download data in advance.  
**Local files**, on the other hand, can reduce initialization time if you already have the data available.

Let's explore both options with these keyword arguments:

In [5]:
surface_forcing_kwargs = {
    "grid": largegrid,
    "start_time": start_time,
    "end_time": end_time,
    "type": "physics",
    "model_reference_date": datetime(2000, 1, 1), # this is the default
    "wind_dropoff": True,
    "correct_radiation": False,
    "use_dask": True
}

### Streaming Cloud-based ERA5 Data

Let's begin by exploring the streaming approach. This method uses [ARCO (Analysis-Ready, Cloud Optimized) ERA5 data](https://github.com/google-research/arco-era5/?tab=readme-ov-file#025-pressure-and-surface-level-data) from the public Google Cloud Storage bucket:
```bash
gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3
```
To activate the streaming approach in ROMS-Tools, simply omit the path in the `source` or explicitly provide the cloud storage URL:

* `source = {"name": "ERA5"}`
* `source = {"name": "ERA5", "path": "gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3"}`

<div class="alert alert-info">

Note

Streaming **requires** `use_dask = True` since the underlying data format is Zarr, which supports lazy loading and chunked access via Dask. [Here](https://roms-tools.readthedocs.io/en/latest/using_dask.html) you can learn more about using `Dask` with `ROMS-Tools`.

</div>

In [ ]:
surface_forcing = SurfaceForcing(
    **surface_forcing_kwargs,
    source={"name": "ERA5"},
)

Instantiating the surface forcing took over 1.5 minutes, even though no computations have been performed yet due to Dask’s lazy evaluation. **This delay results from streaming data from the cloud.** 

The surface forcing variables are held in an `xarray.Dataset` that is accessible via the `.ds` property. Indeed, all variables are Dask arrays representing lazy-loaded data.

In [ ]:
surface_forcing.ds

`ROMS-Tools` has found 505 time stamps within our specified time range. Let's double-check that `ROMS-Tools` has selected the correct times.

In [ ]:
surface_forcing.ds.time

<div class="alert alert-info">

Note

The `time` variable shows relative time, i.e., days since the model reference date (here set to January 1, 2000 by default). The `abs_time` coordinate shows the absolute time. The ERA5 data provided to `ROMS-Tools` has hourly frequency; this temporal frequency is inherited by `surface_forcing`.
    
</div>

To visualize any of the surface forcing fields, we can use the `.plot` method.

In [ ]:
surface_forcing.plot("uwnd", time=0)

In [ ]:
surface_forcing.plot("swrad", time=15)

The spatial dimensions `eta_rho` and `xi_rho` in the surface forcing dataset remain 202, matching their original lengths in the `grid_10km`.

## Saving as NetCDF or YAML file
Once we have decided which of the surface forcing versions we actually want to use, we can save the dataset as a NetCDF file.

The spatial dimensions `eta_rho` and `xi_rho` in the surface forcing dataset remain 202, matching their original lengths in the `grid_10km`.

We need to specify the desired target path.

In [ ]:
surface_forcing.save(os.path.join(outputdir, "croco_frc_nea.nc"), group=True)

In [6]:
surface_forcing_from_local_data = SurfaceForcing(
    **surface_forcing_kwargs,
    source={"name": "ERA5", "path": "/home/ctroupin/data/CROCO/dataERA5.nc"},
)

ValueError: Dataset does not contain all required variables. The following variables are missing: ['sst']